# Tests

In [1]:
import os
import sys
import logging
import requests
import json
import re
import pandas as pd

sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
from lib.nanuq import Nanuq
from lib.gapp import Phenotips
from lib.gapp import BSSH
from lib.samplesheet import SampleSheet

nq    = Nanuq()
pho   = Phenotips()
bssh  = BSSH()

class Args:
    # Simulate an argparse instance
    #
    def __init__(self, run='LH00336_0006', level='info'):
        self.run   = run
        self.level = level

args = Args(level='debug')
vars(args)

{'run': 'LH00336_0006', 'level': 'debug'}

In [2]:
sheet = SampleSheet('D:\\HSJ\\Projects\\PRAGMatIQ\\Runs\\Seq_10B_PRAG_20240130.csv')
sheet.sections['DragenGermline_Data'][0]

['Sample_ID',
 'ReferenceGenomeDir',
 'VariantCallingMode',
 'QcCoverage1BedFile',
 'QcCoverage2BedFile',
 'QcCoverage3BedFile',
 'QcCrossContaminationVcfFile']

In [2]:
sheet = nq.get_samplesheet(args.run)
for line in sheet.text.split('\n'):
    print(line.rstrip())


[
H
e
a
d
e
r
]

F
i
l
e
F
o
r
m
a
t
V
e
r
s
i
o
n
,
2

R
u
n
N
a
m
e
,
L
H
0
0
3
3
6
_
0
0
0
6

I
n
s
t
r
u
m
e
n
t
P
l
a
t
f
o
r
m
,
N
o
v
a
S
e
q

I
n
d
e
x
O
r
i
e
n
t
a
t
i
o
n
,
F
o
r
w
a
r
d


[
R
e
a
d
s
]

R
e
a
d
1
C
y
c
l
e
s
,
1
5
1

R
e
a
d
2
C
y
c
l
e
s
,
1
5
1

I
n
d
e
x
1
C
y
c
l
e
s
,
1
0

I
n
d
e
x
2
C
y
c
l
e
s
,
1
0


[
B
C
L
C
o
n
v
e
r
t
_
S
e
t
t
i
n
g
s
]

S
o
f
t
w
a
r
e
V
e
r
s
i
o
n
,
3
.
8
.
4

F
a
s
t
q
C
o
m
p
r
e
s
s
i
o
n
F
o
r
m
a
t
,
g
z
i
p


[
B
C
L
C
o
n
v
e
r
t
_
D
a
t
a
]

L
a
n
e
,
S
a
m
p
l
e
_
I
D
,
I
n
d
e
x
,
I
n
d
e
x
2
,
S
a
m
p
l
e
_
P
r
o
j
e
c
t

1
,
2
3
3
9
9
,
A
G
A
G
G
C
A
A
C
C
,
C
T
A
A
T
G
A
T
G
G
,
P
R
A
G
M
a
t
I
Q
_
C
H
U
S
J

1
,
2
3
4
0
2
,
C
C
A
T
C
A
T
T
A
G
,
G
G
T
T
G
C
C
T
C
T
,
P
R
A
G
M
a
t
I
Q
_
C
H
U
S
J

1
,
2
3
4
0
4
,
G
A
T
A
G
G
C
C
G
A
,
C
G
C
A
C
A
T
G
G
C
,
P
R
A
G
M
a
t
I
Q
_
C
H
U
S
J

1
,
2
3
4
0
5
,
A
T
G
G
T
T
G
A
C
T
,
G
G
C
C
T
G
T
C
C
T
,
P
R
A
G
M
a
t
I
Q
_
C
H
U
S
J

1
,
2
3
4
0
6
,
T
A
T
T
G
C
G
C
T


In [ ]:
# Get authorization token from Emedgene
# Please note - the Authorization header is only valid for a limited time, and
# expires after 8H. In that case, any request made with an expired token will 
# return a 403 code. To resolve, re-do the Login procedure to get a new token.
#
url      = "https://chusaintejustine.emedgene.com/api/auth/api_login/"
payload  = '{"username": "cqgc.bioinfo.hsj@ssss.gouv.qc.ca", "password": "3175CoteSainte-Catherine"}'
headers  = {'Content-Type': 'application/json'}
response = requests.request("POST", url, headers=headers, data=payload)
auth_header = response.json()["Authorization"]
auth_header


In [ ]:
# Get case for EMG634551172 (GM240123)
#
case_id = "EMG634551172"
get_test_response = requests.get(f'https://chusaintejustine.emedgene.com/api/test/{case_id}/', json={}, headers={'Authorization': auth_header})
# with open('D:\\HSJ\\Projects\\PRAGMatIQ\\emg_scripts\\emg_case_example-GM240123.json', 'w') as fh:
#     json.dump(get_test_response.json(), fh, indent=4)
get_test_response.json().keys()


In [ ]:
# Transfer files to aws
# --profile "emedgene" or "emedgene-eval"
aws s3 ls --profile emedgene-eval s3://cac1-prodca-emg-downloads/Ste_Justine_eval/upload